In [49]:
%load_ext autoreload
%autoreload 2

from load_data.load_data import MPKGraphLoader, TramGraphLoader, BusGraphLoader
from map_utils import get_map, get_hex_area, TransferConfig, get_izochrone_map
import warnings
warnings.filterwarnings('ignore')
import networkx as nx
import matplotlib.pyplot as plt
from qwlist import QList, Lazy
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
loader_2023 = MPKGraphLoader.from_pickle('./data/mpk_graph_loader_2023.pkl')
loader_2024 = MPKGraphLoader.from_pickle('./data/mpk_graph_loader_2024.pkl')

In [47]:
def statistics(graph: nx.Graph):
    strongly_connected = next(nx.strongly_connected_components(graph))
    g = graph.subgraph(strongly_connected)

    nodes_count = graph.number_of_nodes()
    print(f'Nodes count: {nodes_count}')

    edges_count = graph.number_of_edges()
    print(f'Edges count: {edges_count}')

    diameter = nx.diameter(g)
    print(f'Diameter: {diameter}')

    density = nx.density(graph)
    print(f'Density: {density}')

    betweenness_centrality = nx.betweenness_centrality(graph)
    total = len(betweenness_centrality)
    sum_ = Lazy(betweenness_centrality.items()).map(lambda item: item[1]).fold(lambda acc, x: acc + x, 0)
    print(f'Node betweenness centrality: {sum_ / total}')

    edge_betweenness_centrality = nx.edge_betweenness_centrality(graph)
    total = len(edge_betweenness_centrality)
    sum_ = Lazy(edge_betweenness_centrality.items()).map(lambda item: item[1]).fold(lambda acc, x: acc + x, 0)
    print(f'Edge betweenness centrality: {sum_ / total}')

    avg_shortest_path = nx.average_shortest_path_length(g)
    print(f'Avg shortest path: {avg_shortest_path}')

def average_hex_area(loader: MPKGraphLoader, max_time: int, transfer_time: int = 5):
    areas = []
    for name in tqdm(loader.stop_names):
        cfg = TransferConfig(name, max_time, transfer_time)
        area = get_hex_area(8, loader, cfg)
        areas.append(area)
    return sum(areas) / len(areas)


In [43]:
statistics(loader_2023.multigraph)

Nodes count: 2064
Edges count: 13632
Diameter: 57
Density: 0.003201478993112311
Node betweenness centrality: 0.0074095776451867075
Edge betweenness centrality: 0.001194106852131084
Avg shortest path: 16.28673576708089


In [44]:
statistics(loader_2024.multigraph)

Nodes count: 2149
Edges count: 14176
Diameter: 59
Density: 0.0030710225967991696
Node betweenness centrality: 0.007320997725911718
Edge betweenness centrality: 0.0011793300026476184
Avg shortest path: 16.71818211753247


In [50]:
average_hex_area(loader_2023, 30)

100%|██████████| 611/611 [04:38<00:00,  2.20it/s]


56.95259527706332

In [51]:
average_hex_area(loader_2024, 30)

100%|██████████| 622/622 [05:14<00:00,  1.97it/s]


62.38327591486759